# Fedbiomed Researcher to train a federated ppca model.

## Description of the exercise :

Three datasets `n1.csv` , `n2.csv` and `n3.csv` will be generated randomly using 3-views PPCA from a 4-dimensional latent space, with views dimensions [15,8,10] and 2 groups. Henceforth, we will distribute the 3 dataset to 3 distinct nodes and use Fed-mv-PPCA. In each center we check the evolution of expected LL during training.

## Data Generation

We will generate three datasets using mv-PPCA.
Then save them in a path of your choice on your machine.

In [1]:
import numpy as np
import pandas as pd

def sample_x_n(N, q, random_state=None):
    return np.random.RandomState(random_state).randn(N,q)

def generate_data(N_g, W, a_g, mu, sigma2, x_n, view, random_state=None):
    rnd=np.random.RandomState(random_state)

    N=N_g.sum()
    d, q = W.shape
    sigma=np.sqrt(sigma2)

    return compute_mean_likelihood(N_g, W, a_g, mu, x_n, view) + sigma*rnd.randn(N,d)

def compute_mean_likelihood(N_g, W, a_g, mu, x_n, view):
    G=len(N_g)
    N=N_g.sum()
    d, q = W.shape

    g_ind=np.concatenate((np.zeros(1, dtype=np.int64), np.cumsum(N_g)))


    y_n=np.empty((N, d))

    for g in range(G):
        y_n[g_ind[g]:g_ind[g+1]]= np.einsum("dq,nq->nd", W, x_n[g_ind[g]:g_ind[g+1]]+a_g[g]) + mu
    y_n = pd.DataFrame(data=y_n,
                     columns=[f'var_{view},{i + 1}' for i in range(d)])

    return y_n

In [2]:
np.random.seed(100)

D_i = [15, 8, 10]
G = 2
n_centers = 3
testing_samples = 40

q_gen = 4

sigma2_gen1, sigma2_gen2, sigma2_gen3 = 2, 1, 3
W_gen1 = np.random.uniform(-10, 10, (D_i[0], q_gen))
W_gen2 = np.random.uniform(-5, 5, (D_i[1], q_gen))
W_gen3 = np.random.uniform(-15, 15, (D_i[2], q_gen))
mu_gen1 = np.random.uniform(-10, 10, D_i[0])
mu_gen2 = np.random.uniform(-5, 5, D_i[1])
mu_gen3 = np.random.uniform(-15, 15, D_i[2])

a_g_gen = np.concatenate((np.zeros((1, q_gen)), np.random.uniform(-10, 10, (G - 1, q_gen))))

W = [W_gen1,W_gen2,W_gen3]
mu = [mu_gen1,mu_gen2,mu_gen3]
sigma = [sigma2_gen1,sigma2_gen2,sigma2_gen3]

# absent_views contains as key the id of a center in which we want to simulate absent views,
# and as argumet the id of the missing view. 
absent_views = {'2': 2}

for i in range(n_centers):
    N_g = np.array([np.random.randint(25,300),np.random.randint(25,300)])
    g_ind = np.concatenate((np.zeros(1, dtype=np.int64), np.cumsum(N_g)))
    N = N_g.sum()
    x_n_gen = sample_x_n(N, q_gen, random_state=150)
    Y = []
    for d in range(len(D_i)):
        y_t = generate_data(N_g, W[d], a_g_gen, mu[d], sigma[d], x_n_gen, view = d+1, random_state=250)
        if ((str(i+1) in absent_views.keys()) \
            and (type(absent_views[str(i+1)])== int) \
            and (absent_views[str(i+1)]==d+1)):
            absent_views.update({str(i+1): y_t})
            y_abs=pd.DataFrame(np.nan, index = np.arange(N_g.sum()), \
                               columns = [f'var_{d+1},{i + 1}' for i in range(D_i[d])])
            Y.append(y_abs)
        else:
            Y.append(y_t)

    gr = [int(0) for _ in range(N_g[0])]+[int(1) for _ in range(N_g[1])]
    gr = pd.Series(gr)
    Y.append(gr)

    t_i = pd.concat(Y, axis=1)
    t_i.columns.values[-1] = 'Label'
    t_i.to_csv('== Local path to node' + str(i+1) + '.csv',sep=',')
    #np.savetxt('== Local path to node' + str(i+1) + '.csv',t_i,delimiter=',')
               
N_g_test = np.array([testing_samples//2,testing_samples//2])
g_ind_test = np.concatenate((np.zeros(1, dtype=np.int64), np.cumsum(N_g_test)))
N_test = N_g_test.sum()
x_n_gen = sample_x_n(N_test, q_gen, random_state=150)
Y_test = []
for d in range(len(D_i)):
    y_t = generate_data(N_g_test, W[d], a_g_gen, mu[d], sigma[d], x_n_gen, view = d+1, random_state=250)
    Y_test.append(y_t)

gr_test = [0 for _ in range(N_g_test[0])]+[1 for _ in range(N_g_test[1])]
gr_test = pd.Series(gr_test)
Y_test.append(gr_test)

t_test = pd.concat(Y_test, axis=1)

## Start the network and setting the client up
Before running this notebook:
1. You should start the network from fedbiomed-network, as detailed in :
https://gitlab.inria.fr/fedbiomed/fedbiomed

2. You need to configure at least 2 nodes: <br/>
* **Node 1 :** `./scripts/fedbiomed_run node add`
  * Select option 1 to add a csv file to the client
  * Choose the name, tags and description of the dataset (you can write 'sk' always and it will be good)
  * Pick the .csv file you stored the couple X[0],y[0].
  * Check that your data has been added in node 1 by executing `./scripts/fedbiomed_run node list`
  * Run the node using `./scripts/fedbiomed_run node start`. <br/>

* **Node 2 :** Open a second terminal and run ./scripts/fedbiomed_run node add config n2.ini
  * Select option 1 to add a csv file to the client
  * Choose the name, tags and description of the dataset (you can write 'sk' always and it will be good)
  * Pick the .csv file you stored the couple X[1],y[1].
  * Check that your data has been added in node 2 by executing `./scripts/fedbiomed_run node config n2.ini list`
  * Run the node using `./scripts/fedbiomed_run node config n2.ini start`.
  
* **Node 3 :** Open a third terminal and run ./scripts/fedbiomed_run node add config n3.ini
  * Select option 1 to add a csv file to the client
  * Choose the name, tags and description of the dataset (you can write 'sk' always and it will be good)
  * Pick the .csv file you stored the couple X[2],y[2].
  * Check that your data has been added in node 2 by executing `./scripts/fedbiomed_run node config n3.ini list`
  * Run the node using `./scripts/fedbiomed_run node config n3.ini start `.

 Wait until you get `Connected with result code 0`. it means node is online.


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
from fedbiomed.researcher.environ import TMP_DIR
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=TMP_DIR+'/')
model_file = tmp_dir_model.name + '/ppca_id.py'

Hereafter the template of the class you should provide to Fedbiomed :

**__init__** : we add here the needed sklearn libraries
       
**training_data** : you must return here the (X,y) that must be of the same type of 
your method partial_fit parameters. 

In [5]:
%%writefile "$model_file"

from fedbiomed.common.ppca import PpcaPlan
import numpy as np


class IID_MV_PPCA(PpcaPlan):
    def __init__(self, kwargs):
        super(IID_MV_PPCA, self).__init__(kwargs)
        #self.add_dependency(["from sklearn.linear_model import SGDRegressor"])
    
    def training_data(self):
        """
            Perform in this method all data reading and data transformations you need.
            At the end you should provide a tuple (X_obs,Xk,ViewsX,y), where: 
            X_obs is the training dataset, 
            Xk is a list containing the k-specific dataframe if it exists or 'NaN' otherwise,
            ViewsX is the indicator function for observed vies (ViewsX[k]=1 if view k is observed, 0 otherwise)
            y the corresponding labels
            The dataset is normalized using min max scaler if model_args['norm'] is true
            Note: since labels are not needed for the optimization, 
            training_data can also simply return (X_obs,Xk,ViewsX)
            :raise NotImplementedError if developer do not implement this method.
        """
        dataset = pd.read_csv(self.dataset_path,delimiter=',', index_col=0)
        X = dataset.iloc[:,:-1]
        y = dataset[dataset.columns[-1]]
        
        # Xk is a list contianing the view-specific local datasets
        Xk = []
        ViewsX = []
        ind = 0
        for k in range(self.K):
            if X.iloc[:, ind].isnull().values.any():
                Xk.append('NaN')
                ViewsX.append(0)
            else:
                # if norm = true, data are normalized with min max scaler
                X_k = self.normalize_data(X.iloc[:, ind:ind + self.dim_views[k]]) if self.norm \
                    else X.iloc[:, ind:ind + self.dim_views[k]]
                Xk.append(X_k)
                ViewsX.append(1)
            ind += self.dim_views[k]
        
        # The entire dataset is re-built without empty columns
        Xk_obs = [item for item in Xk if type(item) != str]
        X_obs = pd.concat(Xk_obs, axis=1)
        
        return (X_obs,Xk,ViewsX,y)
    

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmpb8ifn8zs/ppca_id.py


**model_args** is a dictionary containing the mv-ppca model arguments: the total number of views across all datasets, the dimension of each view and the latent space size.

**training_args** contains here the number of local iterations for EM. 

In [6]:
tot_views = 3
dim_views = [15, 8, 10]
n_components = 4
norm = True

model_args = {'tot_views': tot_views, 'dim_views': dim_views, 'n_components': n_components, 'norm': norm}

training_args = {'n_iterations': 15}

In [7]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.mlaggregator import MLaggregator

tags =  ['ppca_data']
rounds = 10

# select nodes participing to this experiment
exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='IID_MV_PPCA',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=MLaggregator(),
                 client_selection_strategy=None)

2021-10-11 15:36:16,298 fedbiomed INFO - Messaging researcher_f73a21e9-2868-492a-bcaa-215790d1209d successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x1202a5c40>
2021-10-11 15:36:16,388 fedbiomed INFO - Searching for clients with data tags: ['ppca_data']
2021-10-11 15:36:16,419 fedbiomed INFO - message received:{'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'success': True, 'databases': [{'name': 'ppca_data', 'data_type': 'csv', 'tags': ['ppca_data'], 'description': 'ppca_data', 'shape': [215, 34], 'dataset_id': 'dataset_1e44396b-f70d-4de6-a23d-01c594343d26'}], 'count': 1, 'client_id': 'client_f2103a22-e8ca-4770-ab00-e9eb3b9767af', 'command': 'search'}
2021-10-11 15:36:16,427 fedbiomed INFO - message received:{'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'success': True, 'databases': [{'name': 'ppca_data', 'data_type': 'csv', 'tags': ['ppca_data'], 'description': 'ppca_data', 'shape': [324

In [8]:
# start federated training
exp.run()

2021-10-11 15:36:30,563 fedbiomed INFO - Sampled clients in round 0 ['client_f2103a22-e8ca-4770-ab00-e9eb3b9767af', 'client_e7b1f510-ceb2-4a55-9bdf-f2a54fc3a9a5', 'client_e9aebe88-efbc-4733-927c-581c17b65ba4']
2021-10-11 15:36:30,564 fedbiomed INFO - Send message to client client_f2103a22-e8ca-4770-ab00-e9eb3b9767af - {'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'training_args': {'n_iterations': 15}, 'model_args': {'tot_views': 3, 'dim_views': [15, 8, 10], 'n_components': 4, 'norm': True}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/10/11/my_model_19bef572-ccbc-4a6d-a194-494e6bc2ae87.py', 'params_url': 'http://localhost:8844/media/uploads/2021/10/11/my_model_377db547-8585-41ce-b938-079d1e53139b.pt', 'model_class': 'IID_MV_PPCA', 'training_data': {'client_f2103a22-e8ca-4770-ab00-e9eb3b9767af': ['dataset_1e44396b-f70d-4de6-a23d-01c594343d26']}}
2021-10-11 15:36:30,566 fedbiomed DEBUG 

2021-10-11 15:36:53,199 fedbiomed INFO - message received:{'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'success': True, 'client_id': 'client_e9aebe88-efbc-4733-927c-581c17b65ba4', 'dataset_id': 'dataset_22b9187b-cad3-4eac-9a23-d24ee3de3e8d', 'params_url': 'http://localhost:8844/media/uploads/2021/10/11/node_params_ee9ab368-6570-4f7e-9dad-f8f704e170d2.pt', 'timing': {'rtime_training': 6.669480056999987, 'ptime_training': 4.746079999999999}, 'msg': '', 'command': 'train'}
2021-10-11 15:36:54,918 fedbiomed INFO - message received:{'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'success': True, 'client_id': 'client_e7b1f510-ceb2-4a55-9bdf-f2a54fc3a9a5', 'dataset_id': 'dataset_efebb587-5ca8-4f06-a98c-5b2e3a5df065', 'params_url': 'http://localhost:8844/media/uploads/2021/10/11/node_params_91c189f2-9ec5-4b7a-805b-2d0caae77ac1.pt', 'timing': {'rtime_t

2021-10-11 15:37:16,935 fedbiomed DEBUG - researcher_f73a21e9-2868-492a-bcaa-215790d1209d
2021-10-11 15:37:16,936 fedbiomed INFO - Send message to client client_e7b1f510-ceb2-4a55-9bdf-f2a54fc3a9a5 - {'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'training_args': {'n_iterations': 15}, 'model_args': {'tot_views': 3, 'dim_views': [15, 8, 10], 'n_components': 4, 'norm': True}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/10/11/my_model_19bef572-ccbc-4a6d-a194-494e6bc2ae87.py', 'params_url': 'http://localhost:8844/media/uploads/2021/10/11/researcher_params_a67bbb9c-6e1a-41b3-945c-4d1ebbe8b078.pt', 'model_class': 'IID_MV_PPCA', 'training_data': {'client_e7b1f510-ceb2-4a55-9bdf-f2a54fc3a9a5': ['dataset_efebb587-5ca8-4f06-a98c-5b2e3a5df065']}}
2021-10-11 15:37:16,938 fedbiomed DEBUG - researcher_f73a21e9-2868-492a-bcaa-215790d1209d
2021-10-11 15:37:16,944 fedbiomed INFO - Send message to clie

2021-10-11 15:37:40,149 fedbiomed INFO - message received:{'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'success': True, 'client_id': 'client_e7b1f510-ceb2-4a55-9bdf-f2a54fc3a9a5', 'dataset_id': 'dataset_efebb587-5ca8-4f06-a98c-5b2e3a5df065', 'params_url': 'http://localhost:8844/media/uploads/2021/10/11/node_params_d13fca64-daf2-4d23-9192-f36ed425db21.pt', 'timing': {'rtime_training': 7.565945139999997, 'ptime_training': 6.159245999999996}, 'msg': '', 'command': 'train'}
2021-10-11 15:37:47,340 fedbiomed INFO - Downloading model params after training on client_f2103a22-e8ca-4770-ab00-e9eb3b9767af - from http://localhost:8844/media/uploads/2021/10/11/node_params_aad516ea-c679-4292-8824-148eb233be4b.pt
2021-10-11 15:37:47,392 fedbiomed INFO - Downloading model params after training on client_e9aebe88-efbc-4733-927c-581c17b65ba4 - from http://localhost:8844/media/uploads/2021/10/11/node_params_30b95b67-a0fc-421f-b124

2021-10-11 15:38:02,979 fedbiomed DEBUG - researcher_f73a21e9-2868-492a-bcaa-215790d1209d
2021-10-11 15:38:03,019 fedbiomed INFO - Send message to client client_e9aebe88-efbc-4733-927c-581c17b65ba4 - {'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'training_args': {'n_iterations': 15}, 'model_args': {'tot_views': 3, 'dim_views': [15, 8, 10], 'n_components': 4, 'norm': True}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/10/11/my_model_19bef572-ccbc-4a6d-a194-494e6bc2ae87.py', 'params_url': 'http://localhost:8844/media/uploads/2021/10/11/researcher_params_365b1f79-1979-4263-b18b-fa95d8fa2cb7.pt', 'model_class': 'IID_MV_PPCA', 'training_data': {'client_e9aebe88-efbc-4733-927c-581c17b65ba4': ['dataset_22b9187b-cad3-4eac-9a23-d24ee3de3e8d']}}
2021-10-11 15:38:03,027 fedbiomed DEBUG - researcher_f73a21e9-2868-492a-bcaa-215790d1209d
2021-10-11 15:38:10,301 fedbiomed INFO - message received:{'r

2021-10-11 15:38:33,485 fedbiomed INFO - Downloading model params after training on client_e7b1f510-ceb2-4a55-9bdf-f2a54fc3a9a5 - from http://localhost:8844/media/uploads/2021/10/11/node_params_60a4b7d3-9746-46b2-be94-1493c2e50cb1.pt
2021-10-11 15:38:33,529 fedbiomed INFO - Clients that successfully reply in round 7 ['client_f2103a22-e8ca-4770-ab00-e9eb3b9767af', 'client_e9aebe88-efbc-4733-927c-581c17b65ba4', 'client_e7b1f510-ceb2-4a55-9bdf-f2a54fc3a9a5']
2021-10-11 15:38:33,698 fedbiomed INFO - Sampled clients in round 8 ['client_f2103a22-e8ca-4770-ab00-e9eb3b9767af', 'client_e7b1f510-ceb2-4a55-9bdf-f2a54fc3a9a5', 'client_e9aebe88-efbc-4733-927c-581c17b65ba4']
2021-10-11 15:38:33,700 fedbiomed INFO - Send message to client client_f2103a22-e8ca-4770-ab00-e9eb3b9767af - {'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'training_args': {'n_iterations': 15}, 'model_args': {'tot_views': 3, 'dim_views': [15, 8, 10], 'n_co

2021-10-11 15:38:49,023 fedbiomed DEBUG - researcher_f73a21e9-2868-492a-bcaa-215790d1209d
2021-10-11 15:38:55,599 fedbiomed INFO - message received:{'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'success': True, 'client_id': 'client_f2103a22-e8ca-4770-ab00-e9eb3b9767af', 'dataset_id': 'dataset_1e44396b-f70d-4de6-a23d-01c594343d26', 'params_url': 'http://localhost:8844/media/uploads/2021/10/11/node_params_bcedc63c-bbe3-447d-a0ef-81b3860b2e93.pt', 'timing': {'rtime_training': 6.120714066999994, 'ptime_training': 4.2496069999999975}, 'msg': '', 'command': 'train'}
2021-10-11 15:38:56,260 fedbiomed INFO - message received:{'researcher_id': 'researcher_f73a21e9-2868-492a-bcaa-215790d1209d', 'job_id': 'ba56359f-ae4c-4e19-b4c7-577651b7b8ec', 'success': True, 'client_id': 'client_e9aebe88-efbc-4733-927c-581c17b65ba4', 'dataset_id': 'dataset_22b9187b-cad3-4eac-9a23-d24ee3de3e8d', 'params_url': 'http://localhost:8844/media/u

In [9]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


List the training rounds :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Access the federated params for the last training round :
	- params_path:  /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/researcher_params_8abf4424-8d79-4b6a-98dc-9d5b45b0493e.pt
	- parameter data:  dict_keys(['tilde_muk', 'tilde_Wk', 'tilde_Sigma2k', 'Alpha', 'Beta', 'sigma_til_muk', 'sigma_til_Wk', 'sigma_til_sigma2k'])


## Test

In [ ]:
#exp.aggregated_params[rounds-1]['params']

In [25]:
import numpy as np
import pandas as pd
from numpy.linalg import solve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error

def normalize_data(X):
    """
    This function normalize the dataset X
    :param X: pandas dataframe
    :return pandas dataframe
    """
    col_name = [col.strip() for col in list(X.columns)]
    x = X.values  # returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    norm_dataset = pd.DataFrame(x_scaled, index=X.index, columns=col_name)

    return norm_dataset

def eval_MB(Wk, q, D_i, K, Sigma2, ViewsX):
    """
    Computes matrices M:=inv(I_q+sum_k Wk.TWk/sigma2k) and B:= [W1.T/sigma2K,...,W1.T/sigma2K].
    :param Wk: list of matrices (d_k x q)
    :param Sigma2: list of float > 0
    :return np.arrays
    """
    index = ViewsX.index(1)

    M1 = Wk[index].reshape(D_i[index], q).T.dot(Wk[index].reshape(D_i[index],q)) / Sigma2[index]
    B = Wk[index].reshape(D_i[index], q).T / Sigma2[index]
    for k in range(index + 1, K):
        if ViewsX[k] == 1:
            M1 += Wk[k].reshape(D_i[k], q).T.dot(Wk[k].reshape(D_i[k],q)) / Sigma2[k]
            B = np.concatenate((B, (Wk[k].reshape(D_i[k], q)).T / Sigma2[k]), axis=1)

    M = solve(np.eye(q) + M1,np.eye(q))

    return M, B

def concat_params(park, K, ViewsX):
    """
    This function concatenates parameters from a list
    :param park: list of vectors/matrices to concatenate
    :return np.array
    """
    index = ViewsX.index(1)

    par = park[index]
    for k in range(index + 1, K):
        if ViewsX[k] == 1:
            par = np.concatenate((par, park[k]), axis=0)

    return par

def simu_latent(q,dataset,ViewsX,global_params):
    """
    This function allows sampling of x_n (latent variables) from the posterior distribution 
    (with global parameters).
    :return pandas dataframe
    """
    d = dataset.shape[1]
    N = dataset.shape[0]
    K = len(ViewsX)

    mu = concat_params(global_params['tilde_muk'], K, ViewsX)
    M, B = eval_MB(global_params['tilde_Wk'], q, D_i, K, global_params['tilde_Sigma2k'],ViewsX)

    Xn = [(M.dot(B).dot(dataset.iloc[n].values.reshape(d, 1) - mu)).reshape(1, q) for n in range(N)]

    df = pd.DataFrame(np.vstack(Xn), index=dataset.index)

    return df

def MAE(dataset,ViewsX,q,global_params):
    """
    This function evaluates the MAE using global parameters
    :return float
    """    
    d = dataset.shape[1]
    N = dataset.shape[0]
    K = len(ViewsX)
    
    mu = concat_params(global_params['tilde_muk'], K, ViewsX)
    W = concat_params(global_params['tilde_Wk'], K, ViewsX)
    M, B = eval_MB(global_params['tilde_Wk'], q, D_i, K, global_params['tilde_Sigma2k'],ViewsX)

    T_true = dataset.values.tolist()

    T_pred = []
    for n in range(N):
        Xng = (M.dot(B).dot(dataset.iloc[n].values.reshape(d, 1) - mu)).reshape(q, 1)
        T_pred.append((W.dot(Xng) + mu).reshape(d))

    MAE = mean_absolute_error(T_true, T_pred)

    return MAE

In [26]:
# Test dataset
from sklearn import preprocessing

def test_data(dataset,norm,K,dim_views):
        """
            Equivalent to training_data, for the test dataset
        """
        X = dataset.iloc[:,:-1]
        y = dataset[dataset.columns[-1]]
        
        # Xk is a list contianing the view-specific local datasets
        Xk = []
        ViewsX = []
        ind = 0
        for k in range(K):
            if X.iloc[:, ind].isnull().values.any():
                Xk.append('NaN')
                ViewsX.append(0)
            else:
                # if norm = true, data are normalized with min max scaler
                X_k = normalize_data(X.iloc[:, ind:ind + dim_views[k]]) if norm \
                    else X.iloc[:, ind:ind + dim_views[k]]
                Xk.append(X_k)
                ViewsX.append(1)
            ind += dim_views[k]
        
        # The entire dataset is re-built without empty columns
        Xk_obs = [item for item in Xk if type(item) != str]
        X_obs = pd.concat(Xk_obs, axis=1)
        
        return (X_obs,Xk,ViewsX,y)

In [30]:
global_params = exp.aggregated_params[rounds-1]['params']

######## Train data
MAE_train = []
Latent_Train = pd.DataFrame()
Label_Train = pd.Series()
for c in range(n_centers):
    dataset_c = pd.read_csv('== Local path to node' + str(c+1) + '.csv', delimiter=',', index_col=0)
    X_obs_c,Xk_c,ViewsX_c,y_c = test_data(dataset_c,norm,tot_views,dim_views)
    # Dataframe of latent space for LDA
    Latent_Train = Latent_Train.append(simu_latent(n_components,X_obs_c,ViewsX_c,global_params))
    Label_Train = Label_Train.append(y_c)
    # MAE Train
    MAE_train.append(MAE(X_obs_c,ViewsX_c,n_components,global_params))
    
lda = LinearDiscriminantAnalysis()
X_Train_lda = lda.fit_transform(Latent_Train, Label_Train.reindex(Latent_Train.index))

######## Test data
X_obs_test,Xk_test,ViewsX_test,y_test = test_data(t_test,norm,tot_views,dim_views)

Latent_Test = pd.DataFrame()
Label_Test = pd.Series()

# Dataframe of latent space for LDA
Latent_Test = Latent_Test.append(simu_latent(n_components,X_obs_test,ViewsX_test,global_params))
Label_Test = Label_Test.append(y_test)
# MAE Test
MAE_test = MAE(X_obs_test,ViewsX_test,n_components,global_params)
# LDA Test
Size_tes = Latent_Test.shape[0]
y_pred_test = lda.predict(Latent_Test)
conf_LDA_Test = confusion_matrix(Label_Test.reindex(Latent_Test.index), y_pred_test)
TP = np.diag(conf_LDA_Test)
num_classes = len(np.unique(Label_Test))
accuracy_LDA_Test = sum(TP) / Size_tes

print('MAE train (mean,std) = ({:.6f},{:.6f}) \t MAE test = {:.6f} \t Accuracy in latent space = {:.6f}'.format(np.mean(np.array(MAE_train)), np.std(np.array(MAE_train)), MAE_test, accuracy_LDA_Test))

<ipython-input-30-612ba6b8dc22>:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Label_Train = pd.Series()


MAE train (mean,std) = (0.020350,0.000513) 	 MAE test = 0.035470 	 Accuracy in latent space = 1.000000


<ipython-input-30-612ba6b8dc22>:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Label_Test = pd.Series()
